In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/task-3/CDMC2021_IoTMalware_Train.label
/kaggle/input/task-3/CDMC2021_IoTMalware_Train.data


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
train_cmd = pd.read_csv("../input/task-3/CDMC2021_IoTMalware_Train.data",delimiter=None,names=['Commands'])
train_cmd.head()

,Commands
0,entry0 sym.__uClibc_main;sym._fp_out_narrow sy...
1,entry0 sym.__uClibc_main;sym.resolve_cnc_addr ...
2,entry0 fcn.0804fdeb;main fcn.0804d1a0 fcn.0804...
3,entry0 unk.0x4002ac sym.__uClibc_main;sym._fp_...
4,sym.__udivsi3 sym.__aeabi_idiv0;sym.__aeabi_ui...


In [4]:
train_lbl = pd.read_csv("../input/task-3/CDMC2021_IoTMalware_Train.label",delimiter=None,names=['Labels'])
train_lbl.head()

,Labels
0,1
1,1
2,1
3,1
4,1


In [5]:
train = pd.concat([train_cmd,train_lbl],axis=1)
del train_cmd
del train_lbl
train.head()

,Commands,Labels
0,entry0 sym.__uClibc_main;sym._fp_out_narrow sy...,1
1,entry0 sym.__uClibc_main;sym.resolve_cnc_addr ...,1
2,entry0 fcn.0804fdeb;main fcn.0804d1a0 fcn.0804...,1
3,entry0 unk.0x4002ac sym.__uClibc_main;sym._fp_...,1
4,sym.__udivsi3 sym.__aeabi_idiv0;sym.__aeabi_ui...,1


In [6]:
length = []
for i in range(16521):
  str1 = train.Commands[i]
  str1 = str1.replace(' ','-')
  str1 = str1.replace(';',' ')  
  train.Commands[i] = str1
  list1 = str1.split()
  length.append(len(list1))
train['Length'] = length

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
!pip install eli5

In [8]:
from sklearn.model_selection import KFold
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import mean_squared_log_error
import eli5
from tqdm import tqdm_notebook

In [9]:
def get_features(df):
    df[['Commands', 'Length']]=df[['Commands', 'Length']].astype(str)
    default_preprocessor = CountVectorizer().build_preprocessor()
    def build_preprocessor(field):
        field_idx = list(df.columns).index(field)
        return lambda x: default_preprocessor(x[field_idx])

    vectorizer = FeatureUnion([
        ('Length', CountVectorizer(
            token_pattern='\d+', 
            preprocessor=build_preprocessor('Length'))),
        ('Commands', TfidfVectorizer(
            ngram_range=(2, 5) , sublinear_tf= True,
            preprocessor=build_preprocessor('Commands'), )),
    ])
    x_train_fit = vectorizer.fit(df[['Commands', 'Length']].values)
    x_train_trans = vectorizer.transform(df[['Commands', 'Length']].values)
    
    return vectorizer, x_train_fit, x_train_trans

In [10]:
bias_train = train.drop("Labels", axis=1)
bias_y_train = train.Labels

In [11]:
vectorizer_bias, x_trainfit_bias, x_train_trans_bias = get_features(bias_train)

In [12]:
test_perc_list = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

In [15]:
from sklearn.model_selection import learning_curve
from sklearn.neighbors import KNeighborsClassifier

In [16]:
estimator=KNeighborsClassifier()
cv=5
n_jobs=-1
train_sizes, train_scores, test_scores = learning_curve(estimator,x_train_trans_bias, bias_y_train, cv=cv, n_jobs=n_jobs, train_sizes=test_perc_list)

In [17]:
def get_mean(arr):
    final= []
    for i in range(arr.shape[0]):
        final.append(arr[i].mean())
    return final

In [18]:
trainFinal,testFinal = get_mean(train_scores),get_mean(test_scores)
import plotly.graph_objects as go

In [19]:
fig=go.Figure()
x,y,z = test_perc_list,trainFinal,testFinal
fig.add_trace(go.Scatter(x=x , y=y , name = 'Train Score',line_shape = 'linear'))
fig.add_trace(go.Scatter(x=x , y=z , name = 'Test Score',line_shape = 'linear'))
fig.update_layout(title_text = 'Bais Variance TradeOff')
fig.update_xaxes(title_text='Train Size' , showgrid = False)
fig.update_yaxes(title_text='Accuracy' , showgrid = False)
fig.show()